In [1]:
from importlib.machinery import SourceFileLoader
import pandas as pd
import numpy  as np

wr = SourceFileLoader("mobius", r"../../mobius.py").load_module()
cu = SourceFileLoader("mobius_calib_uncert_lmfit", r"../../mobius_calib_uncert_lmfit.py").load_module()

multi = SourceFileLoader("multiloop", r"../multiloop.py").load_module()
magicopt = SourceFileLoader("magicopt",  r"../magicopt.py").load_module()

In [2]:
wr.initialize('../../../Applications/MAGIC/magic_simple.dll')

parfile = 'templateparameters.dat'
infile  = 'templateinputs.dat'

dataset = wr.DataSet.setup_from_parameter_and_input_files(parfile, infile)

In [3]:
idx = ('Soil', 'Lake')

In [4]:
it = 0
def do_optim(ds, ID, soil, depo) :
    global it
    
    #df = cu.get_input_dataframe(ds, [('SO4 wet deposition scaling factor', [])])
    #df.plot()
    
    
    #print('SO4 wet dep: %f' % ds.get_parameter_double('SO4 wet deposition', []))
    #print('Na wet dep: %f' % ds.get_parameter_double('Na wet deposition', []))
    
    params, comparisons = magicopt.get_base_cations_setup(ds, idx, 'Observed', usemax=True, obsE=True, so4dep=False, no3upt=True)
    
    #params.pretty_print()
    #Set initial guess to the present-day value
    params['ECa_S'].value = soil['ExCa']
    params['EMg_S'].value = soil['ExMg']
    params['ENa_S'].value = soil['ExNa']
    params['EK_S'].value = soil['ExK']
    
    params['WK_S'].min = max(0.0, soil['UptK'] - depo['K'])
    
 
    max_nfev=200

    cu.set_parameter_values(params, ds)
    #ds.write_parameters_to_file('test_params.dat')
    #try:
    res_pars, nfev = magicopt.calib(ds, params, comparisons, method='least_squares', max_nfev=max_nfev)

    if nfev==max_nfev : print('Failed to converge\n')
    #cu.print_goodness_of_fit(ds, comparisons)
    #axes = cu.plot_objective(ds, comparisons)
    #for ax in axes :
    #    ymin, ymax = ax.get_ylim()
    #    ax.set_ylim([0, ymax])      #Set 0 as min of plot y axis

    

    for comparison in comparisons :
        simname, simindexes, obsname, obsindexes, *_ = comparison

        sim = ds.get_result_series(simname, simindexes)
        obs = ds.get_input_series(obsname, obsindexes, alignwithresults=True)

        print('Error of %3s: (sim-obs): %f' % (obsname, np.nansum(sim-obs)))

        
        
    print('\n')

    res_pars.pretty_print()
    print('\n')
    
    #results = cu.get_result_dataframe(ds, [
    #    ('Sum of base cation concentrations (Ca + Mg + Na + K)', ['Soil']),
    #    ('Sum of acid anion concentrations (SO4 + Cl + NO3 + F)', ['Soil']),
    #    ('SO4(2-) ionic concentration', ['Soil']),
    #    ('SO4(2-) ionic concentration', ['Lake']),
    #    ('SO4 deposition', ['Soil']),
    #    ('Cl(-) ionic concentration', ['Soil']),
    #    ('Cl(-) ionic concentration', ['Lake']),
    #    ('NO3(-) ionic concentration', ['Soil']),
    #    ('NO3(-) ionic concentration', ['Lake']),
    #    ('Ca(2+) ionic concentration', ['Soil']),
    #    ('Mg(2+) ionic concentration', ['Soil']),
    #    ('Na(+) ionic concentration', ['Soil']),
    #    ('K(+) ionic concentration', ['Soil']),
    #    ('pH', ['Soil']),
    #])
    #results.plot()
    
    #display(results)

    it+=1

In [5]:
excelfile = 'CCE-%s BJC.xls'

multi.do_magic_loop(dataset, excelfile, do_optim, do_id=15)

#OptPar   CNUP    EMX   WECA   WEMG   WENA    WEK   CN01   EC01   EM01   EN01   EK01   DOC1
 #Value    0.0    0.1   11.7    3.6    5.1    0.5    0.0    3.8    0.8    0.3    1.1  100.0

Running lake HOH 1066 (ID 15)

Error of Observed Ca: (sim-obs): 0.000000
Error of Observed Mg: (sim-obs): 0.000000
Error of Observed Na: (sim-obs): 0.000000
Error of Observed K: (sim-obs): 0.000000
Error of Observed ECa: (sim-obs): 0.000000
Error of Observed EMg: (sim-obs): -0.000000
Error of Observed ENa: (sim-obs): -0.000000
Error of Observed EK: (sim-obs): 0.000000
Error of Observed NO3: (sim-obs): 0.045646


Name          Value      Min      Max   Stderr     Vary     Expr Brute_Step
ECa_S         3.745     0.01    29.46   0.2942     True     None     None
EK_S          1.058     0.01    7.105  0.08413     True     None     None
EMg_S        0.8319     0.01    13.19  0.06553     True     None     None
ENa_S        0.3139     0.01    5.612  0.02512     True     None     None
NO3Sink_S      -100     -100        0    1.016     True     None     None
WCa_S         11.79        0      168   0.9713     True     None     None
WK_S         0.5041        0    25.71  0.04702     True     None

,Sum of base cation concentrations (Ca + Mg + Na + K) [Soil],Sum of acid anion concentrations (SO4 + Cl + NO3 + F) [Soil],SO4(2-) ionic concentration [Soil],SO4(2-) ionic concentration [Lake],SO4 deposition [Soil],Cl(-) ionic concentration [Soil],Cl(-) ionic concentration [Lake],NO3(-) ionic concentration [Soil],NO3(-) ionic concentration [Lake],Ca(2+) ionic concentration [Soil],Mg(2+) ionic concentration [Soil],Na(+) ionic concentration [Soil],K(+) ionic concentration [Soil],pH [Soil]
Date,,,,,,,,,,,,,,
1800-01-01,32.627336,5.570512,0.613031,0.703967,0.040774,4.95748,5.64127,6.261218e-07,0.045860,15.416934,5.661613,10.804438,0.744350,4.1600
1800-02-01,32.653802,5.587744,0.630264,0.706005,0.039843,4.95748,5.64127,3.657918e-07,0.045860,15.432195,5.667208,10.809680,0.744719,4.1600
1800-03-01,32.654366,5.566419,0.608939,0.705617,0.037760,4.95748,5.64127,2.061795e-07,0.045859,15.432545,5.667332,10.809761,0.744728,4.1600
1800-04-01,32.654219,5.570694,0.613214,0.705261,0.040464,4.95748,5.64127,1.166847e-07,0.045859,15.432485,5.667306,10.809701,0.744727,4.1600
1800-05-01,32.654162,5.566014,0.608535,0.704474,0.039770,4.95748,5.64127,6.524898e-08,0.045859,15.432477,5.667300,10.809658,0.744727,4.1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995-08-01,32.510206,8.437696,3.480217,4.148580,0.229043,4.95748,5.64127,7.778327e-13,0.613433,15.418204,5.618591,10.719670,0.753741,4.1525
1995-09-01,32.510918,8.406132,3.448652,4.119788,0.219033,4.95748,5.64127,7.754770e-13,0.611604,15.418148,5.618647,10.720385,0.753738,4.1525
1995-10-01,32.511614,8.387391,3.429912,4.091048,0.226745,4.95748,5.64127,7.728676e-13,0.609704,15.418077,5.618700,10.721103,0.753735,4.1525
